In [81]:
import random
import math
import numpy as np
import time
import os

class LS1:
    def __init__(self, inst, n, weight_list, value_list, weight_threshold_value, random_seed, cut_off):

        # initial the 0-1 pack problem
        self.example_name = os.path.basename(inst)
        self.n = n
        self.weight_list = np.array(weight_list)
        self.value_list = np.array(value_list)
        self.weight_threshold_value = weight_threshold_value
        self.random_seed = random_seed
        self.initial_x = self.Initial_solution()
        self.local_best = self.evaluate(self.initial_x)
        self.cut_off = cut_off
        self.sol_file = " "
        self.trace_file = " "

    def input_size(self):
        if self.n < 50:
            self.sol_file = "LS1_ans\small\{}_LS1_{}_{}.sol".format(self.example_name,self.cut_off,self.random_seed)
            self.trace_file = "LS1_ans\small\{}_LS1_{}_{}.trace".format(self.example_name,self.cut_off,self.random_seed)
        else:
            self.sol_file = "LS1_ans\large\{}_LS1_{}_{}.sol".format(self.example_name,self.cut_off,self.random_seed)
            self.trace_file = "LS1_ans\large\{}_LS1_{}_{}.trace".format(self.example_name,self.cut_off,self.random_seed)

    def evaluate(self,x):
        """
        Caluculate the total value and total weight of one input array x.

        Inputs:
            np.array: x

        Returns:
            list: [totalValue,totalWeight]
        """
        picked = x
        totalValue = np.dot(picked, self.value_list)  # compute the value of the knapsack selection
        totalWeight = np.dot(picked, self.weight_list)  # compute the weight value of the knapsack selection
        return [totalValue,totalWeight]
    
    def flip_neighborhoods(self,x):
        """
        Random pick one neighbourhood.

        Input:
            np.array: x.
        Returns:
            np.array: x_new.
        """
        # change_idx_list = random.sample(range(self.n),round(self.n/2))
        if self.n < 5:
            change_idx_list = random.sample(range(self.n),1)
        else:
            change_idx_list = random.sample(range(self.n),3)
        new_x = np.copy(x)

        for idx in change_idx_list:
            if x[idx] == 1:
                new_x[idx] = 0
            else:
                new_x[idx] = 1

        return new_x

    def Initial_solution(self):
        """
        Initialize the solution using Greedy alg.

        Returns:
            np.array: x.
        """
        # Seeds
        np.random.seed(self.random_seed)
        random.seed(self.random_seed)
        unit_value = [(self.value_list[i]/self.weight_list[i], i) for i in range(self.n)]
        unit_value.sort(reverse=True)

        # Initial the array with greedy method
        x = np.zeros(self.n)
        total_weight = self.weight_threshold_value*0.85

        for unit_value, idx in unit_value:
            if self.weight_list[idx] <= total_weight:
                x[idx] = 1
                total_weight -= self.weight_list[idx]
        return x
    
    # def Initial_solution(self):
    #     """
    #     Initialize the solution.

    #     Returns:
    #         np.array: x.
    #     """
    #     # Seeds
    #     np.random.seed(self.random_seed)
    #     random.seed(self.random_seed)
    #     init_idx_list = random.sample(range(self.n),10)

    #     # Initial the array with greedy method
    #     x = np.zeros(self.n)
    #     for idx in init_idx_list:
    #         x[idx] = 1
    #     while (self.evaluate(x)[1]>self.weight_threshold_value):
    #         init_idx_list = random.sample(range(self.n),10)
    #         x = np.zeros(self.n)
    #         for idx in init_idx_list:
    #             x[idx] = 1

    #     return x

    def simulated_Annealing(self, initial_temp, iter_per_temp, final_temp):
        """
        Run the annealing simulation.

        Inputs:
            int: initial_temp.
            int: iter_per_temp.
            int: final_temp.
        Returns:
            np.array: x.
        """
        # Seeds
        np.random.seed(self.random_seed)
        random.seed(self.random_seed)

        # Make direction
        try:
            os.mkdir("LS1_ans")
        except FileExistsError:
            pass
        
        # Make direction
        try:
            os.mkdir("LS1_ans/large")
            os.mkdir("LS1_ans/small")
        except FileExistsError:
            pass

        start_time = np.float64(time.time())

        self.input_size()

        with open(self.trace_file,"w") as file:
        
            x_curr = self.initial_x  # x_curr will hold the current solution
            x_best = x_curr[:]  # x_best will hold the best solution
            ans_best = self.evaluate(x_best)

            current_end = np.float64(time.time())
            # print(round(current_end-start_time,2),self.local_best[1],self.local_best[0]) #  # print the weight and value
            input_str = str(round(current_end-start_time,2))+" "+str(int(self.local_best[1]))
            file.write(input_str + "\n")
        
            while (initial_temp > final_temp):

                # Time restriction
                current_end = np.float64(time.time())
                if round(current_end-start_time,2) > self.cut_off:
                    # write the current best
                    print(initial_temp," ",self.evaluate(x_best)[1])
                    update_str = str(round(current_end-start_time,2)) + " " + str(int(self.evaluate(x_best)[1]))
                    file.write(update_str + "\n")
                    return x_best

                m = 0 # Counting iteration in current temp
                k = 1 # The constant

                # Iteration in every temperature
                while (m <= iter_per_temp):
                    m += 1

                    # Generate neighbourhood
                    s = self.flip_neighborhoods(x_curr)
                    s_evaluate = self.evaluate(s)

                    # Can't be bigger than the threshold
                    if (s_evaluate[1] <= self.weight_threshold_value):
                        # get a better value
                        if (s_evaluate[0] > ans_best[0]):
                            x_best = s[:]
                            x_curr = s[:]
                            ans_best = s_evaluate[:]
                            current_end = np.float64(time.time())
                            update_str = str(round(current_end-start_time,2)) + " " + str(int(s_evaluate[1]))
                            file.write(update_str + "\n")
                        # Probability
                        else:
                            delta = self.evaluate(x_curr)[0] - self.evaluate(s)[0]
                            change_or_not = random.uniform(0,1)
                            randomness= math.exp(-1 * delta / k / (initial_temp))
                            if (change_or_not < randomness):
                                x_curr=s[:]

                initial_temp = initial_temp * 0.998 # Update the temperature
            
            # reach the final temperature
            current_end = np.float64(time.time())
            final_str = str(round(current_end-start_time,2)) + " " + str(int(ans_best[1]))
            file.write(final_str + "\n")

            return x_best
        # return self.initial_x,x_best

    def save_solution(self,x):
        """
        Write the answer to the file.

        Inputs:
            np.array: x
        """

        best_value = str(self.evaluate(x)[0])
        with open (self.sol_file,"w") as file:
            file.write(best_value + '\n')
            for item in x:
                file.write(str(int(item))+" ")

    def save_solution(self,x):
        """
        Write the answer to the file.

        Inputs:
            np.array: x
        """

        best_value = str(self.evaluate(x)[0])
        with open (self.sol_file,"w") as file:
            file.write(best_value + '\n')
            for item in x:
                file.write(str(int(item))+" ")
    
def load_data(inst):
    values = []
    weights = []
    with open(inst, 'r') as file:
        n, capacity = map(int, file.readline().strip().split())
        for _ in range(n):
            value, weight = map(float, file.readline().strip().split())
            values.append(value)
            weights.append(weight)
    return n,capacity,values, weights

In [97]:
n, capacity, values, weights = load_data("DATA/DATASET/large_scale/large_6")
solution = LS1("/DATA/DATASET/large_scale/large_6", n, weights, values, capacity, 10, 100)
x_best = solution.simulated_Annealing(initial_temp = 100000, iter_per_temp = 1500, final_temp=5)

In [98]:
print(np.dot(x_best,values),np.dot(x_best,weights))

261484.0 24994.0


In [99]:
print(np.dot(solution.initial_x,values),np.dot(solution.initial_x,weights))

255081.0 21262.0


In [9]:
for i in range(1,22):
    print("python starter.py -inst ./DATA/DATASET/large_scale/large_{} -alg LS1 -time 300 -seed 10".format(i))

python starter.py -inst ./DATA/DATASET/large_scale/large_1 -alg LS1 -time 300 -seed 10
python starter.py -inst ./DATA/DATASET/large_scale/large_2 -alg LS1 -time 300 -seed 10
python starter.py -inst ./DATA/DATASET/large_scale/large_3 -alg LS1 -time 300 -seed 10
python starter.py -inst ./DATA/DATASET/large_scale/large_4 -alg LS1 -time 300 -seed 10
python starter.py -inst ./DATA/DATASET/large_scale/large_5 -alg LS1 -time 300 -seed 10
python starter.py -inst ./DATA/DATASET/large_scale/large_6 -alg LS1 -time 300 -seed 10
python starter.py -inst ./DATA/DATASET/large_scale/large_7 -alg LS1 -time 300 -seed 10
python starter.py -inst ./DATA/DATASET/large_scale/large_8 -alg LS1 -time 300 -seed 10
python starter.py -inst ./DATA/DATASET/large_scale/large_9 -alg LS1 -time 300 -seed 10
python starter.py -inst ./DATA/DATASET/large_scale/large_10 -alg LS1 -time 300 -seed 10
python starter.py -inst ./DATA/DATASET/large_scale/large_11 -alg LS1 -time 300 -seed 10
python starter.py -inst ./DATA/DATASET/la

In [10]:
math.log(5/10000,0.996)

1896.4226249414528

In [103]:
with open("large_21_LS1_300_10.sol", "r") as file:
    lines = file.readlines()

# 提取第一行数字
first_line = lines[0].strip()

# 提取第二行的0和1组成的字符串，并按空格分割
second_line = lines[1].strip().split()

# 输出第一行
print(first_line)

# 输出1对应的序号
for index, value in enumerate(second_line, start=1):
    if value == '1':
        print(index)



140748.0
13
21
30
75
86
90
97
107
114
121
148
158
164
165
170
205
212
234
243
244
266
269
272
274
293
295
303
308
324
344
376
392
423
424
433
473
476
480
484
491
499
506
525
534
539
547
563
564
575
579
584
585
598
638
660
664
666
669
676
695
722
725
731
740
758
759
765
768
834
839
843
850
853
856
884
896
910
922
927
939
947
953
959
960
987
989
1002
1008
1079
1098
1101
1106
1147
1150
1160
1168
1171
1187
1188
1190
1194
1197
1199
1206
1207
1221
1225
1262
1268
1274
1287
1303
1310
1315
1339
1345
1346
1352
1354
1370
1377
1380
1386
1407
1412
1417
1437
1452
1464
1465
1468
1472
1473
1475
1487
1492
1541
1551
1556
1562
1568
1573
1600
1606
1615
1616
1618
1624
1634
1638
1645
1649
1657
1658
1661
1677
1680
1682
1690
1691
1696
1699
1711
1732
1734
1760
1775
1835
1836
1850
1854
1875
1878
1891
1900
1946
1969
1979
1985
1994
2004
2007
2014
2026
2044
2046
2052
2063
2068
2072
2079
2087
2097
2115
2120
2142
2148
2150
2158
2160
2172
2180
2192
2193
2199
2229
2235
2238
2242
2273
2287
2289
2300
2307
2312
2316
2317

In [110]:
import numpy as np

# 假设arr是一个全为0和1的NumPy数组
arr = np.array([0, 1, 0, 1, 1, 0, 1])

# 找到值为1的索引
indices = np.where(arr == 1)[0]

# 将索引转换为字符串，并用逗号隔开
index_str = ','.join(map(str, indices))

print(index_str)

2,4,5,7
